# kaggle - Titanic: Machine Learning from Disaster

下の2つのページを参考にしながら実施  
u++さん:  
・https://www.kaggle.com/sishihara/hypothesis-and-visualization-for-titanic-in-kaggle/notebook  
  
KaggleチュートリアルTitanicで上位3%以内に入るには。(0.82297):  
・https://lp-tech.net/articles/0QUUd

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re as re # 正規表現ライブラリ

## データセットの読み込み

In [3]:
train = pd.read_csv('./input/train.csv')    # 891 x 12
test = pd.read_csv('./input/test.csv')    #418 x 11

### データセットを連結して特徴量エンジニアリングしやすくする

In [4]:
data = train.append(test) # testデータのSurvivedはNaNになる
data.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


## 特徴量エンジニアリング

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


### 1. pclass
Passenger Class  
・Null値なし  
・int64型→マッピング必要なし

### 2. Sex
性別データ  
・Null値なし  
・object型→マッピングが必要   
  
マッピング  
男性：0  
女性：1

In [6]:
# inplaceするので2回目以降の実行はエラーとなる
train['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
test['Sex'].replace(['male', 'female'], [0, 1], inplace=True)

### 3. Embarked
 乗船した港　Cherbourg、Queenstown、Southamptonの３種類  
`Embarked       1307 non-null object`  
・Null値が2つある→埋める必要あり  
・object型→マッピングが必要  

参考①では再頻出のS(Southampton)で埋めている  
そのほかの埋め方で考えられるのは以下  
・Null値が多い場合は'C','Q','S'の出現頻度の確率で埋めても良いかもしれない  
・家族関係がわかれば、同じ港で乗ったことにする方法もある  

In [7]:
# まずはNull値を'S'で埋める
train['Embarked'].fillna(('S'), inplace=True)
test['Embarked'].fillna(('S'), inplace=True)

# object型の変数をintに変換
train['Embarked'] = train['Embarked'].map({'S':0, 'C':1, 'Q':2}).astype(int);
test['Embarked'] = test['Embarked'].map({'S':0, 'C':1, 'Q':2}).astype(int);

### 4. Fare
運賃  
`Fare           1308 non-null float64`  
・Null値が2つある→埋める必要あり  
・float64型→マッピングの必要なし  

参考①では全体の平均値で埋めている  
そのほかの埋め方で考えられるの以下  
・家族関係がわかれば、同じ運賃とすることができる

In [8]:
train['Fare'].fillna(np.mean(train['Fare']), inplace=True)
test['Fare'].fillna(np.mean(train['Fare']), inplace=True)# testデータは使わず、trainデータを使って平均を埋める

運賃を4つのカテゴリに分類する

In [9]:
train['Categorical_Fare'] = pd.qcut(train['Fare'], 4, labels=False)# 量をもとにビン分割
test['Categorical_Fare'] = pd.qcut(train['Fare'], 4, labels=False)# testデータは使わず、trainデータを使って分割する

### 5. Age
年齢  
`Age            1046 non-null float64`  
・Null値が結構ある　→　何らかの方法で埋める必要あり  
・float64型→マッピング必要なし  

参考①では±σの範囲内の乱数で値を埋めている。  
値が出そろったところで5カテゴリに分類する

In [14]:
age_ave = train['Age'].mean()
age_std = train['Age'].std()

train['Age'].fillna(np.random.randint(age_ave - age_std, age_ave + age_std), inplace=True)
test['Age'].fillna(np.random.randint(age_ave - age_std, age_ave + age_std), inplace=True)

年齢を5つのカテゴリに分類する

In [15]:
train['Categorical_Age'] = pd.cut(train['Age'], 5, labels=False)
test['Categorical_Age'] = pd.cut(train['Age'], 5, labels=False)

### 6. Name
名前  
`Name           1309 non-null object`
・Null値なし  
・object型→マッピングが必要

